<a href="https://colab.research.google.com/github/abhishekverma0207/abhishekverma_ljmu/blob/main/M5/M5_Databuildup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'

In [13]:
!kaggle competitions download -c m5-forecasting-accuracy

 55% 25.0M/45.8M [00:00<00:00, 119MB/s] 
100% 45.8M/45.8M [00:00<00:00, 148MB/s]


In [15]:
!rm *.zip

# Data loading

#### Installing dependent packages

In [ ]:
!pip install lightning
!pip install pytorch_forecasting

In [11]:
import pandas as pd
import numpy as np


# import lightning.pytorch as pl
# from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
# from lightning.pytorch.loggers import TensorBoardLogger
# import numpy as np
# import pandas as pd
import torch
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet